### Student Information
Name: 戴天吟

Student ID: 111034551

GitHub ID: Charlotte038

Kaggle name: Charlotte

Kaggle private scoreboard snapshot: 

[Snapshot1](w2v_xgb.png)
[Snapshot2](Kaggle_Leaderboard.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the DM2023-Lab2-master. You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 27th 11:59 pm, Wednesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 31th 11:59 pm, Sunday)__. 

In [ ]:
### Begin Assignment Here

### Take Home Exercises

In [ ]:
### Begin Assignment Here

import pandas as pd

### training data
anger_train = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/data/semeval/train/anger-ratings-0to1.train.txt",
                         sep="\t", header=None,names=["id", "text", "emotion", "intensity"])
sadness_train = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-master/data/semeval/train/sadness-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_train = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-master/data/semeval/train/fear-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_train = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-master/data/semeval/train/joy-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
# combine 4 sub-dataset
train_df = pd.concat([anger_train, fear_train, joy_train, sadness_train], ignore_index=True)
### testing data
anger_test = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/data/semeval/dev/anger-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
sadness_test = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/data/semeval/dev/sadness-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_test = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/data/semeval/dev/fear-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_test = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/data/semeval/dev/joy-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

# combine 4 sub-dataset
test_df = pd.concat([anger_test, fear_test, joy_test, sadness_test], ignore_index=True)
# shuffle dataset
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)
print("Shape of Training df: ", train_df.shape)
print("Shape of Testing df: ", test_df.shape)

---
#### ** Exercise 1 (Take home): **  
Plot word frequency for Top 30 words in both train and test dataset. (Hint: refer to DM lab 1)


In [ ]:
# Answer here
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt

# Assuming you have loaded your train and test data into train_data and test_data respectively

# Tokenize and count words in train data
train_words = word_tokenize(' '.join(train_df))
train_word_freq = Counter(train_words)
train_top30 = train_word_freq.most_common(30)

# Tokenize and count words in test data
test_words = word_tokenize(' '.join(test_df))
test_word_freq = Counter(test_words)
test_top30 = test_word_freq.most_common(30)

def plot_word_frequency(top_words, title):
    words, counts = zip(*top_words)
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(words)), counts, align='center')
    plt.yticks(range(len(words)), words)
    plt.gca().invert_yaxis()
    plt.xlabel('Frequency')
    plt.title(title)
    plt.show()

# Plot top 30 words in train data
plot_word_frequency(train_top30, 'Top 30 Words in Train Data')

# Plot top 30 words in test data
plot_word_frequency(test_top30, 'Top 30 Words in Test Data')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize CountVectorizer and fit on train data
BOW_vectorizer = CountVectorizer(tokenizer=nltk.word_tokenize)
BOW_vectorizer.fit(train_df['text'])

# Transform train and test data to Bag-of-Words features
train_data_BOW_features = BOW_vectorizer.transform(train_df['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_df['text'])

# Get feature names
feature_names = BOW_vectorizer.get_feature_names()

# Sum up word occurrences in train and test data
train_term_count = np.asarray(train_data_BOW_features.sum(axis=0)).ravel()
test_term_count = np.asarray(test_data_BOW_features.sum(axis=0)).ravel()

# Create DataFrames for top 30 words and their frequencies in train and test data
df_train = pd.DataFrame({'count': train_term_count, 'terms': feature_names})
df_train = df_train.sort_values('count', ascending=False)[:30]

df_test = pd.DataFrame({'count': test_term_count, 'terms': feature_names})
df_test = df_test.sort_values('count', ascending=False)[:30]

# Plotting top 30 word frequencies in Train and Test data
fig, axs = plt.subplots(2, 1, figsize=(10, 13))
axs[0].set_title('Top 30 word frequency in Train data')
axs[1].set_title('Top 30 word frequency in Test data')

# Using seaborn barplot for visualization
sns.barplot(x='terms', y='count', data=df_train, ax=axs[0])
sns.barplot(x='terms', y='count', data=df_test, ax=axs[1])

# Rotate x-axis labels for better readability
for ax in axs:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

plt.tight_layout()
plt.show()


---
#### ** Exercise 2 (Take home): **  
Generate an embedding using the TF-IDF vectorizer instead of th BOW one with 1000 features and show the feature names for features [100:110].

In [ ]:
# Answer here
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df['text'])

# Get feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Display feature names for features 100 to 110
print("Feature names for features 100 to 110:")
print(feature_names[100:111])


---
#### ** Exercise 3 (Take home): **  
Can you interpret the results above? What do they mean?

In [ ]:
# Answer here
'''
# 1.Accuracy
training accuracy: 0.99
testing accuracy: 0.68
-> training accuracy very high but testing accuracy is low -> overfitting 

# 2.precision, recall, f1-score
Precision (精確率)：Precision表示模型在預測某個類別時的準確性，即在所有被分類為某一類別的樣本中，真正屬於該類別的比例。
-> 高精確率表示模型將正確地識別出更多相應的類別
Recall (召回率)：Recall表示模型成功識別出特定類別樣本的能力，即在所有屬於某一類別的樣本中，成功被模型分類為該類別的比例。
-> 高召回率表示模型能夠更好地捕捉到該類別的所有實際樣本
F1-score：F1-score是Precision和Recall的調和平均數，它綜合考慮了Precision和Recall，通常用於綜合評估模型的表現。
-> F1-score越高，表示模型在Precision和Recall之間取得了良好的平衡
-> 從結果來看，'joy'具有 high precision, high recall and high F1-score，表現最佳

# 3.check by confusion matrix
混淆矩陣的對角線(從左上到右下的元素)表示模型成功預測的數量(True Positive)，而非對角線的元素表示模型預測錯誤的情況(False Positive和False Negative)
->
第一行代表猜測為'anger'的結果。有57個樣本實際結果為'anger'，表示模型將這57個樣本正確地預測(True Positive)
第二行代表猜測為'fear'的結果。有76個True Positive，其餘為False Positive和False Negative
第三行代表猜測為'joy'的結果。有56個True Positive，其餘為False Positive和False Negative
第四行代表猜測為'sadness'的結果。有47個True Positive，其餘為False Positive和False Negative

'''

---
#### ** Exercise 4 (Take home): **  
Build a model using a ```Naive Bayes``` model and train it. What are the testing results? 

*Reference*: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
# Answer here
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load the dataset (example with 20 Newsgroups dataset)
data_train = fetch_20newsgroups(subset='train', shuffle=True)
data_test = fetch_20newsgroups(subset='test', shuffle=True)

# Vectorize the text data
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_train.data)
X_test = tfidf_vectorizer.transform(data_test.data)

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, data_train.target)

# Predict on the test set
y_test_pred = nb_classifier.predict(X_test)

# Evaluate the model
print(classification_report(data_test.target, y_test_pred))


---
#### ** Exercise 5 (Take home): **  

How do the results from the Naive Bayes model and the Decision Tree model compare? How do you interpret these differences? Use the theoretical background covered in class to try and explain these differences.

In [ ]:
# Answer here
# pros & cons, which situation use Naive Bayes model is better? And which situation use Decision Tree model is better?

'''
Naive Bayes vs. Decision Tree:

Performance Metrics:
NB: Naive Bayes classifiers work well on text data, especially in natural language processing tasks like sentiment analysis or document classification. They tend to be fast and require relatively small amounts of training data. However, they assume feature independence which might not hold true in some cases.
DT: Decision Trees are versatile and can handle a variety of data types, providing interpretable rules. They can capture non-linear relationships between features and the target variable. However, they can easily overfit and may not generalize well if the tree becomes too deep.

Interpretability:
NB: Naive Bayes models are relatively simple and offer high interpretability. The probabilities they calculate for each class can be interpreted as likelihoods given the observed features.
DT: Decision Trees provide explicit rules for classification, making them easy to interpret and visualize. Each split in the tree represents a decision based on a specific feature.

Dataset Characteristics:
NB: Naive Bayes assumes feature independence, which might not be valid for all datasets. If the features are correlated, it might affect the model's performance.
DT: Decision Trees can handle complex relationships between features, but they can overfit if the tree is too deep. They might struggle with datasets where the decision boundaries are not aligned with feature axes.

Interpretation based on Theoretical Background:
Naive Bayes: The NB model assumes that features are conditionally independent given the class. This assumption might hold well for text classification tasks (e.g., spam detection), where words might be somewhat independent of each other given the class label (though this assumption is often violated to some extent).
Decision Trees: DTs partition the feature space based on feature values, making decisions based on splitting criteria. They can capture complex relationships but might create overly complex trees that don't generalize well.

Comparing Results:
Performance: The NB model might perform well on text data due to its simplicity and the nature of text features, while Decision Trees might perform better if the relationships between features are more complex or non-linear.
Interpretability: Both models offer interpretability, but Decision Trees provide explicit rules, which can be more easily understood by humans.

The choice between NB and DT models depends on the nature of the dataset, the complexity of relationships between features, interpretability requirements, and the trade-off between accuracy and model complexity. Conducting a detailed analysis considering these aspects would provide deeper insights into the differences observed in their performances.
'''


---
#### ** Exercise 6 (Take home): **  

Plot the Training and Validation Accuracy and Loss (different plots), just like the images below.(Note: the pictures below are an example from a different model). How to interpret the graphs you got? How are they related to the concept of overfitting/underfitting covered in class?
<table><tr>
    <td><img src="pics/pic3.png" style="width: 300px;"/> </td>
    <td><img src="pics/pic4.png" style="width: 300px;"/> </td>
</tr></table>

In [ ]:
# Answer here
import matplotlib.pyplot as plt

training_log = pd.DataFrame()
training_log = pd.read_csv("C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/logs/training_log.csv")
training_log

# Plot training & validation accuracy values
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(training_log['accuracy'])
plt.plot(training_log['val_accuracy'])
plt.title('Training Accuracy per epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train accuracy', 'Validation accuracy'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(training_log['loss'])
plt.plot(training_log['val_loss'])
plt.title('Training Loss per epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train loss', 'Validation loss'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
'''
Overfitting:
Training vs. Validation Accuracy: Initially, both accuracies rise. However, if the training accuracy continues to increase while validation accuracy plateaus or decreases, it suggests overfitting. The model is memorizing the training data but fails to generalize to new data.
Training vs. Validation Loss: A divergence between the training loss decreasing while the validation loss increases or remains stagnant indicates overfitting. The model learns the noise in the training data instead of the underlying patterns.

Underfitting:
Low Training and Validation Accuracy: Both accuracies remain low, suggesting the model is too simple to capture patterns in the data.
High Training and Validation Loss: Elevated loss values across both training and validation datasets signify underfitting. The model struggles to learn relevant patterns.

Relation to Overfitting/Underfitting:
Overfitting: When the training performance (accuracy/loss) is significantly better than the validation performance, it suggests the model has memorized the training data too well, resulting in poor generalization.
Underfitting: Occurs when both training and validation performances are subpar, indicating the model's inability to capture patterns in the data, possibly due to simplicity or poor training.

-> this case is overfitting.
'''

---
#### ** Exercise 7 (Take home): **  

Now, we have the word vectors, but our input data is a sequence of words (or say sentence). 
How can we utilize these "word" vectors to represent the sentence data and train our model?


In [ ]:
# Answer here
# 1. Averaging Word Vectors:
# Method: Calculate the average of word vectors for all words in the sentence.
import numpy as np
from gensim.models import KeyedVectors

model_path = "GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

def average_word_vectors(words, model, num_features):
    word_vecs = []
    for word in words:
        if word in model:
            word_vecs.append(model[word])
    if len(word_vecs) == 0:
        return np.zeros(num_features)
    word_vecs = np.array(word_vecs)
    return np.mean(word_vecs, axis=0)

# Example usage
sentence = "This is an example sentence"
words = sentence.split()
sentence_vector = average_word_vectors(words, w2v_google_model, 300)  # Assuming 300-dimensional word vectors


In [ ]:
# 2. TF-IDF Weighted Word Vectors:
# Method: Compute TF-IDF (Term Frequency-Inverse Document Frequency) weighted average of word vectors.
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 假設你有一個包含句子的列表 sentences
sentences = ["This is an example sentence.", "Another example sentence."]

# 初始化 TF-IDF 向量化器
tfidf_vectorizer = TfidfVectorizer()

# 將文本轉換為 TF-IDF 特徵
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

# 獲取詞彙表和詞彙到索引的映射
vocab = tfidf_vectorizer.vocabulary_

# 獲取 TF-IDF 加權詞向量
def get_tfidf_weighted_vectors(sentence, model, tfidf_matrix, vocab):
    words = sentence.split()
    word_vecs = []
    weights = []
    for word in words:
        if word in model and word in vocab:
            word_vec = model[word]
            word_idx = vocab[word]
            tfidf_weight = tfidf_matrix[0, word_idx]  # Assuming only one document in tfidf_matrix
            word_vecs.append(word_vec * tfidf_weight)
            weights.append(tfidf_weight)
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    weighted_avg = np.average(word_vecs, axis=0, weights=weights)
    return weighted_avg

# 獲取 TF-IDF 加權句子向量
sentence = "This is an example sentence."
tfidf_weighted_vector = get_tfidf_weighted_vectors(sentence, w2v_google_model, tfidf_matrix, vocab)


In [ ]:
# 3. Doc2Vec or Paragraph Vectors:
# Method: Train models like Doc2Vec or paragraph vectors to directly learn fixed-size representations for sentences.
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# 假設你有一個包含句子的列表 sentences
tagged_data = [TaggedDocument(words=sentence.split(), tags=[str(i)]) for i, sentence in enumerate(sentences)]

# 初始化 Doc2Vec 模型
doc2vec_model = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=100)

# 建立詞彙表並訓練模型
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# 獲取句子向量
sentence_vector = doc2vec_model.infer_vector("This is an example sentence.".split())


---
#### ** Exercise 8 (Take home): **  

Generate a t-SNE visualization to show the 15 words most related to the words "angry", "happy", "sad", "fear" (60 words total).

In [ ]:
# Answer here
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Words of interest
target_words = ["angry", "happy", "sad", "fear"]

# Number of similar words to consider for each target word
num_similar_words = 15

# Initialize lists to store similar words and their vectors
similar_words = []
word_vectors = []

# Retrieve similar words and their vectors for each target word
for word in target_words:
    # Get similar words and their similarity scores
    similar = w2v_google_model.most_similar(word, topn=num_similar_words)
    for item in similar:
        similar_words.append(item[0])
        word_vectors.append(w2v_google_model[item[0]])

# t-SNE model
tsne = TSNE(n_components=2, metric='cosine', random_state=42)

# Transform word vectors using t-SNE
X_tsne = tsne.fit_transform(word_vectors)

# Plot the t-SNE visualization
plt.figure(figsize=(10, 8))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], marker='o', c='blue')

# Annotate each point with its respective word
for i, word in enumerate(similar_words):
    plt.annotate(word, xy=(X_tsne[i, 0], X_tsne[i, 1]), xytext=(5, 2),
                 textcoords='offset points', ha='right', va='bottom', fontsize=8)

plt.title('t-SNE Visualization of Similar Words')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.grid(True)
plt.show()


### Kaggle Competition

The implementation steps of my Kaggle competition project are as follows:
1. Load Data
- This part of code performs three essential data processing steps: data loading, data merging, and data splitting. 
- Initially, I retrieve data from distinct source files, such as JSON and CSV files containing information about tweets, emotion labels, and data identification. 
- Next, I merge these datasets based on a common feature, like tweet IDs, creating a more comprehensive dataset to facilitate deeper insights into relationships within the data. 
- Finally, I divide the integrated dataset into training and testing sets based on identification labels, enabling the subsequent use of this data for training and evaluating machine learning models.


2. Save Data
- I save the data in Pickle format. 
- The pickle module implements binary protocols for serializing and de-serializing a Python object structure.


3. Data Preprocessing
- This code segment focuses on text preprocessing for Natural Language Processing (NLP) tasks.
- The focus here is to prepare text data by removing noise (special characters, punctuation, stopwords) and standardizing it (lowercasing) for better analysis and model training in NLP tasks. 
- This preparation helps in extracting meaningful patterns from the text data.


4. Other Feature Engineering
- BoW + TF-IDF
    - The teacher mentioned during class that Bog+TF-IDF might have good results.
    - The significance lies in converting raw text data into numerical format (vectors) suitable for machine learning models.
    - CountVectorizer creates a numerical representation of text, while TF-IDF further refines it by considering the importance of words in each document relative to the entire dataset.
- Word2Vec
    - The focus here is on converting words in tweets into continuous numerical representations (word embeddings) using Word2Vec, enabling machine learning models to understand the semantic meaning and context of words within the text data. 
    - This transformation allows for the utilization of word embeddings in various downstream tasks such as sentiment analysis or classification.


5. Model 
- Support Vector Machine (SVM) model
    - I initially utilized the CPU for the execution, but the processing time was excessively prolonged in generating results. Considering this, I might opt to switch to Colab for more efficient performance.
    - SVC is a popular choice for classification tasks, especially when dealing with binary and multi-class classification problems. Its ability to handle non-linear data through kernel functions and its effectiveness in high-dimensional spaces make it suitable for sentiment classification tasks where distinguishing between multiple classes (emotions, in this case) is required. The 'linear' kernel specified here indicates a linear separation of classes in the feature space.
- Decision Tree
    - Also the processing time was excessively prolonged in generating results.
- Logistic Regression
    - [Snapshot3](Logistic_Regression.png)
    - Logistic Regression is a classic linear model commonly used for binary classification tasks. 
    - It's efficient, interpretable, and works well for problems with linearly separable classes. 
    - In sentiment analysis, it can be effective when dealing with binary (positive/negative) or multi-class sentiment classification tasks.
- Random Forest
    - [Snapshot4](Random_Forest.png)
    - Random Forest is an ensemble learning method that builds multiple decision trees and merges their predictions to improve accuracy and reduce overfitting.
- Word2Vec+XGBoost (Best Performance)
    - [Snapshot5](w2v_xgb.png)
    - XGBoost is an effective and popular gradient boosting algorithm known for its high performance in classification tasks, providing strong predictive power and handling non-linear relationships between features and targets effectively. When combined with word embeddings, it can yield robust results in sentiment analysis by leveraging the semantics embedded in the word vectors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.corpus import stopwords
import regex as re
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Load Data

In [2]:
# Load the data
tweets = pd.read_json('C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/Kaggle Competition/dm2023-isa5810-lab2-homework/tweets_DM.json', lines=True)
emotion_labels = pd.read_csv('C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/Kaggle Competition/dm2023-isa5810-lab2-homework/emotion.csv')
data_identification = pd.read_csv('C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/Kaggle Competition/dm2023-isa5810-lab2-homework/data_identification.csv')

# tweets = pd.read_json('C:/Users/flyin/OneDrive/桌面/HW/112-1/資料探勘/DMLab2/Kaggle Competition/dm2023-isa5810-lab2-homework/tweets_DM.json', lines=True)
# emotion_labels = pd.read_csv('C:/Users/flyin/OneDrive/桌面/HW/112-1/資料探勘/DMLab2/Kaggle Competition/dm2023-isa5810-lab2-homework/emotion.csv')
# data_identification = pd.read_csv('C:/Users/flyin/OneDrive/桌面/HW/112-1/資料探勘/DMLab2/Kaggle Competition/dm2023-isa5810-lab2-homework/data_identification.csv')

In [3]:
tweets

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets
...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets


In [4]:
import json
# Load the data from tweets_DM.json
id_list = []
text_list = []
score_list = []

with open('C:/Users/user/Desktop/碩士課程資料/112-1/資料探勘/HW/DM2023-Lab2-Master/Kaggle Competition/dm2023-isa5810-lab2-homework/tweets_DM.json', 'r') as f:
    into_list = f.readlines()
    for data_json in into_list:
        data = json.loads(data_json)
        score_list.append(data['_score'])
        id_list.append(data['_source']['tweet']['tweet_id'])
        text_list.append(data['_source']['tweet']['text'])

data_tweet = {"tweet_id": id_list, "text": text_list, "score": score_list}
tweets = pd.DataFrame(data_tweet)

In [5]:
tweets

,tweet_id,text,score
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",391
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",433
2,0x28b412,"Confident of your obedience, I write to you, k...",232
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,376
4,0x2de201,"""Trust is not the same as faith. A friend is s...",989
...,...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,827
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,368
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,498
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",840


In [6]:
emotion_labels

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [7]:
data_identification

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [8]:
# Merge datasets
merged_data = pd.merge(tweets, emotion_labels, on='tweet_id')

In [9]:
# Merge datasets
merged_data=pd.merge(tweets,data_identification,on='tweet_id',how='outer')
merged_data= pd.merge(merged_data,emotion_labels,on='tweet_id',how= 'outer')
merged_data.head()  

,tweet_id,text,score,identification,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",391,train,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",433,train,sadness
2,0x28b412,"Confident of your obedience, I write to you, k...",232,test,NaN
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,376,train,fear
4,0x2de201,"""Trust is not the same as faith. A friend is s...",989,test,NaN


In [10]:
# split train and test data by its' identification

df_train = merged_data[merged_data['identification']=='train']
df_test = merged_data[merged_data['identification']=='test']

In [11]:
df_train

,tweet_id,text,score,identification,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",391,train,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",433,train,sadness
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,376,train,fear
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,120,train,joy
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,1021,train,anticipation
...,...,...,...,...,...
1867526,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,94,train,joy
1867527,0x38959e,In every circumtance I'd like to be thankful t...,627,train,joy
1867528,0x2cbca6,there's currently two girls walking around the...,274,train,joy
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",840,train,joy


In [12]:
df_test

,tweet_id,text,score,identification,emotion
2,0x28b412,"Confident of your obedience, I write to you, k...",232,test,NaN
4,0x2de201,"""Trust is not the same as faith. A friend is s...",989,test,NaN
9,0x218443,When do you have enough ? When are you satisfi...,66,test,NaN
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",104,test,NaN
33,0x26289a,"In these tough times, who do YOU turn to as yo...",310,test,NaN
...,...,...,...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th...",602,test,NaN
1867529,0x2a980e,"""There is a lad here, which hath five barley l...",598,test,NaN
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,827,test,NaN
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,368,test,NaN


#### Save Data

In [13]:
## save to pickle file
df_train.to_pickle("df_train.pkl") 
df_test.to_pickle("df_test.pkl")

In [14]:
# Loading the DataFrame from pickle file
df_train = pd.read_pickle("df_train.pkl")
df_test = pd.read_pickle("df_test.pkl")

#### Data Pre-processing

In [15]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back into a single string
    clean_text = ' '.join(filtered_tokens)
    
    return clean_text

# Apply preprocessing to 'text' column in the df_train DataFrame
df_train['clean_text'] = df_train['text'].apply(preprocess_text)

# Apply preprocessing to 'text' column in the df_test DataFrame
df_test['clean_text'] = df_test['text'].apply(preprocess_text)

# Displaying the updated DataFrame with the cleaned text for df_train
print(df_train[['text', 'clean_text', 'emotion']].head())

# Displaying the updated DataFrame with the cleaned text for df_test
print(df_test[['text', 'clean_text', 'emotion']].head())  # Ensure 'emotion' is included for verification


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0  People who post "add me on #Snapchat" must be ...   
1  @brianklaas As we see, Trump is dangerous to #...   
3                Now ISSA is stalking Tasha 😂😂😂 <LH>   
5  @RISKshow @TheKevinAllison Thx for the BEST TI...   
6       Still waiting on those supplies Liscus. <LH>   

                                          clean_text       emotion  
0  people post add snapchat must dehydrated cuz m...  anticipation  
1  brianklaas see trump dangerous freepress aroun...       sadness  
3                             issa stalking tasha lh          fear  
5  riskshow thekevinallison thx best time tonight...           joy  
6                   still waiting supplies liscus lh  anticipation  
                                                 text  \
2   Confident of your obedience, I write to you, k...   
4   "Trust is not the same as faith. A friend is s...   
9   When do you have enough ? When are you satisfi...   
30  God woke you up, 

#### Other Feature engineering

##### BoW+TF-IDF

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Create CountVectorizer for df_train
count_vectorizer_train = CountVectorizer(max_features=1000)
bow_matrix_train = count_vectorizer_train.fit_transform(df_train['text'])

# Convert BoW to TF-IDF representation for df_train
tfidf_transformer_train = TfidfTransformer()
X_train = tfidf_transformer_train.fit_transform(bow_matrix_train)
y_train = df_train['emotion']

# Apply the same transformation to df_test using the fitted transformers from df_train
bow_matrix_test = count_vectorizer_train.transform(df_test['text'])
X_test = tfidf_transformer_train.transform(bow_matrix_test)
y_test = df_test['emotion']


In [26]:
X_train

<1455563x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 13350564 stored elements in Compressed Sparse Row format>

In [27]:
y_train

0          anticipation
1               sadness
3                  fear
5                   joy
6          anticipation
               ...     
1867526             joy
1867527             joy
1867528             joy
1867533             joy
1867534             joy
Name: emotion, Length: 1455563, dtype: object

In [24]:
X_test

<411972x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 4672243 stored elements in Compressed Sparse Row format>

In [40]:
y_test

2          NaN
4          NaN
9          NaN
30         NaN
33         NaN
          ... 
1867525    NaN
1867529    NaN
1867530    NaN
1867531    NaN
1867532    NaN
Name: emotion, Length: 411972, dtype: object

##### Word2Vec

In [41]:
from gensim.models import Word2Vec
import numpy as np

# Tokenize the text into words (assuming 'text' is the preprocessed text)
tokenized_tweets_train = df_train['text'].apply(lambda x: x.split())
tokenized_tweets_test = df_test['text'].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_tweets_train, vector_size=100, window=5, min_count=1, workers=4)

# Function to get vector representation of a tweet
def get_tweet_vector(tweet_tokens):
    tweet_vector = np.zeros((word2vec_model.vector_size,), dtype="float32")
    num_words = 0
    for word in tweet_tokens:
        if word in word2vec_model.wv:
            tweet_vector = np.add(tweet_vector, word2vec_model.wv[word])
            num_words += 1
    if num_words != 0:
        tweet_vector = np.divide(tweet_vector, num_words)
    return tweet_vector

# Create vectors for each tweet in train and test sets
tweet_vectors_train = tokenized_tweets_train.apply(lambda x: get_tweet_vector(x))
tweet_vectors_test = tokenized_tweets_test.apply(lambda x: get_tweet_vector(x))

# Convert tweet vectors to arrays
X_train = np.array(tweet_vectors_train.tolist())
X_test = np.array(tweet_vectors_test.tolist())

# Define target variable 'emotion'
y_train = df_train['emotion']
y_test = df_test['emotion']

#### Model

##### Support Vector Machine (SVM) model

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Handling missing values in 'emotion' column for df_test
df_test['emotion'].fillna('unknown', inplace=True)  # Filling NaNs with a default category like 'unknown'

# Encode emotions to numerical labels for df_train
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(df_train['emotion'])

# Define the classifier (SVM)
svm_classifier = SVC(kernel='linear', C=1, probability=True)

# Fit the classifier on the training data
svm_classifier.fit(X_train, y_train_encoded)

# Predict on the test set (df_test)
y_pred = svm_classifier.predict(X_test)

# Decode the predicted labels back to emotions
predicted_emotions = label_encoder.inverse_transform(y_pred)

# Create a DataFrame with 'tweet_id' and 'predicted_emotion'
submission_df = pd.DataFrame({'tweet_id': df_test['tweet_id'], 'predicted_emotion': predicted_emotions})

# Save the submission file
submission_df.to_csv('submission_SVC.csv', index=False)


##### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a model and train - Decision Tree Classifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

##### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Using Logistic Regression model
log_reg_classifier = LogisticRegression(max_iter=1000)

# Encode the target variable 'y_train'
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Training the model on the training data
log_reg_classifier.fit(X_train, y_train_encoded)

# Performing predictions on the test set
predicted_emotions = log_reg_classifier.predict(X_test)

# Decoding the predicted labels
predicted_emotions = label_encoder.inverse_transform(predicted_emotions)

# Adding predicted emotions to the df_test DataFrame
df_test['predicted_emotion'] = predicted_emotions

# Creating the submission file 'submission.csv'
submission_df = df_test[['tweet_id', 'predicted_emotion']]
submission_df.to_csv('submission_Logistic Regression.csv', index=False)


##### Random Forest

In [2]:
from sklearn.ensemble import RandomForestClassifier  # Import Random Forest Classifier

# Using Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters like n_estimators

# Encode the target variable 'y_train'
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Training the model on the training data
rf_classifier.fit(X_train, y_train_encoded)

# Performing predictions on the test set
predicted_emotions = rf_classifier.predict(X_test)

# Decoding the predicted labels
predicted_emotions = label_encoder.inverse_transform(predicted_emotions)

# Adding predicted emotions to the df_test DataFrame
df_test['predicted_emotion'] = predicted_emotions



KeyboardInterrupt



In [34]:
# Creating the submission file 'submission.csv'
submission_df = df_test[['tweet_id', 'predicted_emotion']]
submission_df.to_csv('submission_RandomForest.csv', index=False)

In [35]:
df_test_RF=df_test.copy()

##### Word2Vec + XGBoost

In [39]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Tokenize the text into words (assuming 'clean_text' is the preprocessed text)
tokenized_tweets_train = df_train['clean_text'].apply(lambda x: x.split())
tokenized_tweets_test = df_test['clean_text'].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_tweets_train, vector_size=100, window=5, min_count=1, workers=4)

# Function to get vector representation of a tweet
def get_tweet_vector(tweet_tokens):
    tweet_vector = np.zeros((word2vec_model.vector_size,), dtype="float32")
    num_words = 0
    for word in tweet_tokens:
        if word in word2vec_model.wv:
            tweet_vector = np.add(tweet_vector, word2vec_model.wv[word])
            num_words += 1
    if num_words != 0:
        tweet_vector = np.divide(tweet_vector, num_words)
    return tweet_vector

# Create vectors for each tweet in train and test sets
tweet_vectors_train = tokenized_tweets_train.apply(lambda x: get_tweet_vector(x))
tweet_vectors_test = tokenized_tweets_test.apply(lambda x: get_tweet_vector(x))

# Convert tweet vectors to arrays
X_train = np.array(tweet_vectors_train.tolist())
X_test = np.array(tweet_vectors_test.tolist())

# Extracting the target variable 'emotion' from df_train
y_train = df_train['emotion']

# Encoding the target variable 'y_train'
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Define a classifier (for example, XGBoost)
xgb_classifier = XGBClassifier()

# Fit the classifier on the training data using tweet vectors
xgb_classifier.fit(X_train, y_train_encoded)

# Predict on the test set
y_pred = xgb_classifier.predict(X_test)

# Decode the predicted labels back to emotions
predicted_emotions = label_encoder.inverse_transform(y_pred)

# Add predicted emotions to the df_test DataFrame
df_test['predicted_emotion_w2v_xgb'] = predicted_emotions

# Create the submission file 'submission_w2v_xgb.csv'
submission_w2v_xgb_df = df_test[['tweet_id', 'predicted_emotion_w2v_xgb']]
submission_w2v_xgb_df.to_csv('submission_w2v_xgb.csv', index=False)
